In [510]:
import os
from typing import Tuple, List
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, recall_score, f1_score
from sklearn.datasets import fetch_openml
import pennylane as qml
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import time
import matplotlib.pyplot as plt
import seaborn as sns
from plot_utils import PlotUtils
import csv
from datetime import datetime
from pen import QuantumClassifier
from pen_hilbert import QuantumHilbertClassifier
import random
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [511]:
def prepare_data_multiclass(
    data_dir: str = "data/dataset_v2/Training/",
    image_size: int = 128,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Carga, redimensiona y etiqueta imágenes de las 4 clases para clasificación multiclase.
    Realiza downsampling para balancear las clases (todas tendrán el mismo número de imágenes que la clase minoritaria).
    Guarda un log de las imágenes seleccionadas en results/graphics/downsampling_log.txt.
    Devuelve (X, y) como arrays de numpy.
    """
    import random
    import os
    random.seed(seed)
    class_map = {
        "glioma_tumor": 0,
        "meningioma_tumor": 1,
        "pituitary_tumor": 2,
        "no_tumor": 3
    }
    files_by_class = {}
    for class_name in class_map:
        class_dir = os.path.join(data_dir, class_name)
        files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if f.lower().endswith('.jpg')]
        files_by_class[class_name] = files
    # Downsampling: encontrar la clase minoritaria
    min_count = min(len(files) for files in files_by_class.values())
    print("la clase menoritaria tiene", min_count, "imágenes")
    selected_files_log = {}
    for class_name in files_by_class:
        files = files_by_class[class_name]
        random.shuffle(files)
        selected = []
        step = 10
        block = 5
        for i in range(0, len(files), step):
            selected.extend(files[i:i+block])
        selected = selected[:min_count]  # Por si se pasa del límite
        files_by_class[class_name] = selected
        selected_files_log[class_name] = selected
    # Guardar log
    os.makedirs('results/logs', exist_ok=True)
    with open('results/logs/downsampling_log.txt', 'w') as f:
        for class_name, files in selected_files_log.items():
            f.write(f"{class_name} ({len(files)}):\n")
            for file in files:
                f.write(f"    {file}\n")
            f.write("\n")
    X, y = [], []
    for class_name, label in class_map.items():
        for f in files_by_class[class_name]:
            img = Image.open(f).convert('L').resize((image_size, image_size))
            X.append(np.array(img))
            y.append(label)
    X = np.stack(X)
    y = np.array(y)
    return X, y 

In [512]:

class QuantumClassifier:
    def __init__(self, n_qubits=8, pca_features=8, batch_size=16, epochs=20, lr=0.01, layers=3, seed=42):
        self.n_qubits = n_qubits
        self.pca_features = pca_features
        self.batch_size = batch_size
        self.epochs = epochs
        self.lr = lr
        self.layers = layers
        self.seed = seed
        torch.manual_seed(seed)
        np.random.seed(seed)
        self._prepare_data_custom()
        self._build_model()

    def _prepare_data_custom(self):
        X, y = prepare_data_multiclass()
        X = X.reshape((X.shape[0], -1)) / 255.0  # flatten and normalize
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        pca = PCA(n_components=self.pca_features)
        X_pca = pca.fit_transform(X_scaled)
        
        # Añadir este escalado
        scaler_angle = MinMaxScaler(feature_range=(0, np.pi / 2))
        X_pca_scaled = scaler_angle.fit_transform(X_pca)

        # Usar X_pca_scaled para el split
        X_train, X_test, y_train, y_test = train_test_split(X_pca_scaled, y, test_size=0.2, random_state=self.seed)
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = np.array(y_train)
        self.y_test = np.array(y_test)

    def _build_model(self):
        #dev = qml.device("default.qubit", wires=self.n_qubits)
        dev = qml.device("lightning.qubit", wires=self.n_qubits) # Sugerido
        def circuit(inputs, weights):
            #for i in range(self.n_qubits):
            #   qml.RY(inputs[i], wires=i)
            #qml.templates.StronglyEntanglingLayers(weights, wires=range(self.n_qubits))
            qml.AngleEmbedding(inputs, wires=range(self.n_qubits), rotation='Y')
            qml.AngleEmbedding(inputs, wires=range(self.n_qubits), rotation='Z')
            # Usa un ansatz más simple y menos capas
            qml.templates.BasicEntanglerLayers(weights, wires=range(self.n_qubits))
            return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]

        #weight_shapes = {"weights": (self.layers, self.n_qubits, 3)}
        weight_shapes = {"weights": (1, self.n_qubits)}  # <- CORRECTO

        qlayer = qml.qnn.TorchLayer(qml.qnode(dev)(circuit), weight_shapes)

        class HybridModel(nn.Module):
            def __init__(self, qlayer, n_classes=4, n_qubits_model=None, input_features=None):
                super().__init__()
                self.fc_input = nn.Linear(input_features, n_qubits_model)  # <- corregido
                self.qlayer = qlayer
                self.bn1 = nn.BatchNorm1d(n_qubits_model)
                self.hidden1 = nn.Linear(n_qubits_model, 64)
                self.dropout1 = nn.Dropout(0.3)
                self.hidden2 = nn.Linear(64, 32)
                self.dropout2 = nn.Dropout(0.2)
                self.output = nn.Linear(32, n_classes)
                self.relu = nn.ReLU()

            def forward(self, x):
                x = self.fc_input(x)
                x = self.qlayer(x)
                x = self.bn1(x)
                x = self.relu(self.hidden1(x))
                x = self.dropout1(x)
                x = self.relu(self.hidden2(x))
                x = self.dropout2(x)
                return self.output(x)

        self.model = HybridModel(qlayer, n_classes=4, n_qubits_model=self.n_qubits, input_features=self.pca_features)


       

    def train_and_evaluate(self):
        X_train_t = torch.tensor(self.X_train, dtype=torch.float32)
        y_train_t = torch.tensor(self.y_train, dtype=torch.long)
        train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=self.batch_size, shuffle=True)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        loss_fn = nn.CrossEntropyLoss()
        scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)
        loss_history = []
        start = time.time()
        for epoch in range(self.epochs):
            for xb, yb in train_loader:
                pred = self.model(xb)
                loss = loss_fn(pred, yb)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            loss_history.append(loss.item())
            print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")
                # <- Aquí va el scheduler
            scheduler.step(loss.item())
            for param_group in optimizer.param_groups:
                print("Current LR:", param_group['lr'])
            
        end = time.time()

        with torch.no_grad():
            X_test_t = torch.tensor(self.X_test, dtype=torch.float32)
            y_test_t = torch.tensor(self.y_test, dtype=torch.long)
            preds = self.model(X_test_t)
            preds_cls = torch.argmax(preds, dim=1)
            acc = (preds_cls == y_test_t).float().mean().item()
            cm = confusion_matrix(y_test_t.cpu().numpy(), preds_cls.cpu().numpy())
            recall = recall_score(y_test_t.cpu().numpy(), preds_cls.cpu().numpy(), average='macro')
            f1 = f1_score(y_test_t.cpu().numpy(), preds_cls.cpu().numpy(), average='macro')

        class_names = ['glioma', 'meningioma', 'pituitary', 'no_tumor']
        PlotUtils.plot_loss(loss_history, save_path='results/graphics/loss_function_pen.png')
        PlotUtils.plot_confusion_matrix(cm, class_names=class_names, save_path='results/graphics/confusion_matrix_pen.png')

        results = {
            'epochs': self.epochs,
            'learning_rate': self.lr,
            'features': self.pca_features,
            'layers': self.layers,
            'batch_size': self.batch_size,
            'loss': float(loss.item()),
            'accuracy': acc,
            'recall': recall,
            'f1_score': f1,
            'confusion_matrix': cm.tolist(),
            'execution_time': end - start
        }
        print(f"Accuracy: {acc:.4f}")
        print("Confusion Matrix:\n", cm)
        print(f"Recall: {recall:.4f}")
        print(f"F1 score: {f1:.4f}")
        return results


In [513]:
class ExperimentRunner:
    def __init__(self, epochs, lr, features, layers, batch_size, seed):
        self.epochs = epochs
        self.lr = lr
        self.features = features
        self.layers = layers
        self.batch_size = batch_size
        self.seed = seed

    def run_and_log(self, results, csv_file):
        duration = results.get('execution_time', None)
        date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        header = [
            'date', 'execution_time', 'epochs', 'learning_rate', 'features', 'layers', 'batch_size',
            'loss', 'accuracy', 'recall', 'f1_score'
        ]
        row = [
            date, f'{duration:.2f}', results['epochs'], results['learning_rate'], results['features'],
            results['layers'], results['batch_size'], results['loss'], results['accuracy'], results['recall'], results['f1_score']
        ]
        file_exists = os.path.isfile(csv_file)
        with open(csv_file, 'a', newline='') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(header)
            writer.writerow(row)
        print('Results saved in', csv_file)
        print('Run summary:')
        print(row)


In [514]:
class QuantumRunner(ExperimentRunner):
    def run(self):
        print("\n--- Running QUANTUM QuantumClassifier ---")
        qc = QuantumClassifier(
            n_qubits=self.features,
            pca_features=self.features,
            batch_size=self.batch_size,
            epochs=self.epochs,
            lr=self.lr,
            layers=self.layers,
            seed=self.seed
        )
        unique_classes, counts = np.unique(qc.y_train, return_counts=True)
        print("Distribution of classes in training data (y_train):")
        for cls, count in zip(unique_classes, counts):
            print(f"  Class {cls}: {count} samples")
        unique_classes_y, counts_y = np.unique(qc.y_test, return_counts=True)
        print("Distribution of classes in training data (y_train):")
        for cls, counts_y in zip(unique_classes_y, counts_y):
            print(f"  Class {cls}: {counts_y} samples")
        start_time = time.time()
        results = qc.train_and_evaluate()
        end_time = time.time()
        self.duration = end_time - start_time
        results['execution_time'] = self.duration
        self.run_and_log(results, 'results_log.csv')

In [515]:
# =====================
# CONFIGURATION CONSTANTS
# =====================
MODE = 'quantum'  # Options: 'quantum', 'quantum_hilbert', 'both'
EPOCHS = 30
# CONFIGURACIÓN SUGERIDA
LEARNING_RATE = 0.005 # O incluso 0.001
FEATURES = 8
LAYERS = 1
BATCH_SIZE = 32
SEED = 42
USE_HILBERT = True  # Only relevant for quantum

In [516]:
QuantumRunner(EPOCHS, LEARNING_RATE, FEATURES, LAYERS, BATCH_SIZE, SEED).run()


--- Running QUANTUM QuantumClassifier ---
la clase menoritaria tiene 2000 imágenes
Distribution of classes in training data (y_train):
  Class 0: 791 samples
  Class 1: 787 samples
  Class 2: 806 samples
  Class 3: 816 samples
Distribution of classes in training data (y_train):
  Class 0: 209 samples
  Class 1: 213 samples
  Class 2: 194 samples
  Class 3: 184 samples
Epoch 1: Loss = 1.1495
Current LR: 0.005
Epoch 2: Loss = 1.1069
Current LR: 0.005
Epoch 3: Loss = 0.9022
Current LR: 0.005
Epoch 4: Loss = 1.0337
Current LR: 0.005
Epoch 5: Loss = 0.9816
Current LR: 0.005
Epoch 6: Loss = 1.0234
Current LR: 0.005
Epoch 7: Loss = 0.8172
Current LR: 0.005
Epoch 8: Loss = 0.9069
Current LR: 0.005
Epoch 9: Loss = 1.0911
Current LR: 0.005
Epoch 10: Loss = 0.7304
Current LR: 0.005
Epoch 11: Loss = 0.9516
Current LR: 0.005
Epoch 12: Loss = 0.9630
Current LR: 0.005
Epoch 13: Loss = 0.9789
Current LR: 0.005
Epoch 14: Loss = 0.7826
Current LR: 0.005
Epoch 15: Loss = 1.1332
Current LR: 0.005
Epoch 1